In [2]:
from milnlp.converters.pdf_to_text import PdfConverter, create_text

In [154]:
# filepath = "pdf_examples/normal_document.pdf"
# filepath = "pdf_examples/powerpoint.pdf"
# filepath = "pdf_examples/glossary.pdf" 
filepath = "pdf_examples/challenge.pdf"

converter = PdfConverter()
data = converter.convert_pdf(filepath, to_file=False)

### Step 1 - Break sentences on \n\n and remove \n

In [170]:
import re

pattern = re.compile(r'\D([!?.]|[!?.]["”])[ \n]')  # pattern for separating sentences

raw_paragraphs = data.split('\n\n')
# Remove PDF newline artifacts
document = [None]*len(raw_paragraphs)
cleaned_paragraphs = [None]*len(raw_paragraphs)
for ii, raw_paragraph in enumerate(raw_paragraphs):
    cleaned_paragraphs[ii] = raw_paragraph.replace("\n", "")
    
    # Split up normal structured sentences based on the idea of punctuation followed by a space or newline. 
    query = pattern.finditer(cleaned_paragraphs[ii])
    
    start = 0
    sentences = []
    for match in query:
        end, next_start = match.span()
        raw_sentence = cleaned_paragraphs[ii][start:end]+match[0][:-1]  # up to index -1 because the match could be '. ' or '." '
        # Clean up common sentence issues
        sentence = raw_sentence.strip(' ')  # extra leading/trailing spaces
        # Todo migrate utilities from codebase to simple implementations here
        # remove weblinks
        # remove headings (maybe) re =~ (\d+\w?[.):-])+ +  Link: regexr.com/41pqg
        # remove short lines (maybe)
        # remove unlikely sentences (not enough words or something)
        # remove non-ascii (i.e. unicode and hex which can happen in weird cases)
        
        # Add sentence to paragraph
        sentences.append(sentence)  # append the punctuation so that the sentence is still complete
        start = next_start
        
    # Do something with sentences todo improve
    document[ii] = sentences
    
# Remove empty paragraphs from document  todo later we may want to use multiple newlines (adjacent empty paragraphs) to infer natural sections of text
document = [paragraph for paragraph in document if paragraph] 

In [171]:
for sentence in document[7]:
    print(sentence, '\n')

Operation  of  multi-crew  aircraft  with  variably  reduced  onboard  crew  is  a  considerable challenge. 

Multiple crew positions are generally implemented when the operating workload exceeds the reasonable capability of a single operator or when mission safety considerations mandate  the  redundancy  of  additional  crew  members. 

Interactions  of  skilled  multi-crew operator teams can be highly complex. 

At the same time, there is considerable potential payoff to robustly achieving the vision of variably crewed operations, including reduced training and operating  costs,  and  freeing  aircraft  operators  from  low  level  tasks  to  enhance  their  overall mission capability. 



In [145]:
document

[['• SOO Req.'],
 ['SOO Req.', '1SOO Req.', '2SOO Req.', '3SOO Req.'],
 ['integration services for the work.']]

In [147]:
len(data)

4139

Powerpoint files are almost always aready summaries of topics, and the information they convey is often inadequate without the context of a "slide" or heading from the section of a slide. Thus, it does not seem practical or feasible to extract out sentences from the powerpoint and attempt to score their value without the reamining slide context. It is also not feasible to implement a rapid solution to extract out chunks of text from slides based on their context due to formatting issues with PDF. The task of using information within PPT files should be delegated to future efforts to parse the PPT file directly. 

In [167]:
# Test for PPT
threshhold = 0.3

flagged = 0
for paragraph in cleaned_paragraphs:  # breaks if paragraph is empty
    paragraph = paragraph.strip(' ')
    if paragraph[0] in {'-', '•', ''}:
        flagged += 1

print(f"{flagged}/{len(cleaned_paragraphs)} flagged as presentation elements")
if flagged/len(cleaned_paragraphs) > threshhold:
    print("Skipping file due to lack of substantial natural language text.")

IndexError: string index out of range